In [0]:
# ipmorting Libraries
from pyspark.sql.functions import *
import pandas as pd
from pyspark.sql import window,Window
window = Window.orderBy("subject")

In [0]:
#reading file
df= spark.read.option("multiline","true").json("dbfs:/FileStore/JSON_files/example_2.json")

In [0]:
df.printSchema()

root
 |-- quiz: struct (nullable = true)
 |    |-- maths: struct (nullable = true)
 |    |    |-- q1: struct (nullable = true)
 |    |    |    |-- answer: string (nullable = true)
 |    |    |    |-- options: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- question: string (nullable = true)
 |    |    |-- q2: struct (nullable = true)
 |    |    |    |-- answer: string (nullable = true)
 |    |    |    |-- options: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- question: string (nullable = true)
 |    |-- sport: struct (nullable = true)
 |    |    |-- q1: struct (nullable = true)
 |    |    |    |-- answer: string (nullable = true)
 |    |    |    |-- options: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- question: string (nullable = true)



In [0]:
#extracting subject 
df1 =df.select(df["quiz.maths"].alias("maths"),df["quiz.sport"].alias("sport"))

In [0]:
df1.printSchema()

root
 |-- maths: struct (nullable = true)
 |    |-- q1: struct (nullable = true)
 |    |    |-- answer: string (nullable = true)
 |    |    |-- options: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- question: string (nullable = true)
 |    |-- q2: struct (nullable = true)
 |    |    |-- answer: string (nullable = true)
 |    |    |-- options: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- question: string (nullable = true)
 |-- sport: struct (nullable = true)
 |    |-- q1: struct (nullable = true)
 |    |    |-- answer: string (nullable = true)
 |    |    |-- options: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- question: string (nullable = true)



In [0]:
dfquestion=df1.select(df1["maths.q1.question"].alias("question1"),df1["maths.q2.question"].alias("question2"))
#question

In [0]:

#maths dataframe
df2=df1.select(df1["maths.q1.question"].alias("question"))
df2=df2.withColumn("subject",lit("maths"))
df3=df1.select(df1["maths.q2.question"].alias("question"))
df3=df3.withColumn("subject",lit("maths"))

dfmaths=df2.union(df3)

#sports daataframe
dfsport=df1.select(df1["sport.q1.question"].alias("question"))
dfsport=dfsport.withColumn("subject",lit("sport"))

#question dataframe
dfq=dfmaths.union(dfsport)
dfq=dfq.withColumn("QNo",lit(row_number().over(window)))
colorder=["QNo","subject","question"]
dfquestion=dfq.select(*colorder)
dfquestion.show()




+---+-------+--------------------+
|QNo|subject|            question|
+---+-------+--------------------+
|  1|  maths|           5 + 7 = ?|
|  2|  maths|          12 - 8 = ?|
|  3|  sport|Which one is corr...|
+---+-------+--------------------+



In [0]:
dfmaths=df1.select(df1["maths.q1.options"].alias("options"))
dfmaths2=df1.select(df1["maths.q2.options"].alias("options"))
dfsports=df1.select(df1["sport.q1.options"].alias("options"))
 

#options dataframe
dfOptions=dfmaths.union(dfmaths2).union(dfsports)
dfOptions=dfOptions.withColumn("ONo",row_number().over(Window.orderBy("options")))
dfOptions.show()  


+--------------------+---+
|             options|ONo|
+--------------------+---+
|        [1, 2, 3, 4]|  1|
|    [10, 11, 12, 13]|  2|
|[New York Bulls, ...|  3|
+--------------------+---+



In [0]:
# answers dataframe
dfm1=df1.select(df1["maths.q1.answer"].alias("answer"))
dfm1=dfm1.withColumn("ANo",lit("1"))

dfm2=df1.select(df1["maths.q2.answer"].alias("answer"))
dfm2=dfm2.withColumn("ANo",lit("2"))

dfs=df1.select(df1["sport.q1.answer"].alias("answer"))
dfs=dfs.withColumn("ANo",lit("3"))
dfA=dfm1.union(dfm2).union(dfs)
dfA.show()

+-------------+---+
|       answer|ANo|
+-------------+---+
|           12|  1|
|            4|  2|
|Huston Rocket|  3|
+-------------+---+



In [0]:
dfFinal=dfquestion.join(dfOptions,dfquestion.QNo==dfOptions.ONo,"inner").join(dfA,dfquestion.QNo==dfA.ANo,"inner")

FinalColumns=['QNo', 'subject', 'question','A','B','C','D', 'answer']
dfFinal=dfFinal.select(*['QNo', 'subject', 'question', 'options', 'answer'])

dfFinal=dfFinal.withColumn("A",dfFinal.options[0])\
    .withColumn("B",dfFinal.options[1])\
    .withColumn("C",dfFinal.options[2])\
    .withColumn("D",dfFinal.options[3])

dfFinal=dfFinal.select(*FinalColumns)
dfFinal.show()

+---+-------+--------------------+--------------+-----------------+--------------------+-------------+-------------+
|QNo|subject|            question|             A|                B|                   C|            D|       answer|
+---+-------+--------------------+--------------+-----------------+--------------------+-------------+-------------+
|  1|  maths|           5 + 7 = ?|             1|                2|                   3|            4|           12|
|  2|  maths|          12 - 8 = ?|            10|               11|                  12|           13|            4|
|  3|  sport|Which one is corr...|New York Bulls|Los Angeles Kings|Golden State Warr...|Huston Rocket|Huston Rocket|
+---+-------+--------------------+--------------+-----------------+--------------------+-------------+-------------+



In [0]:
#writng File in CSV
dfFinal.write.option("header",True).csv("dbfs:/FileStore/output/FinaldataExam")